# 📊 05_evaluation.ipynb - Evaluating Thematic and Emotional Labels and Translation Mapping

This notebook provides tools to explore, validate, and visualize the labels assigned to Bible verses during emotion and theme classification. It will also visualize the Spanish version.

## 🧱 1. Setup Paths & Translation Maps

In [ ]:
from pathlib import Path
import pandas as pd

BIBLE = "bible_kjv"
BIBLE_ES = "bible_rv60"

EN_DIR = Path("data/labeled") / BIBLE / "emotion_theme"
ES_DIR = Path("data/labeled") / BIBLE_ES / "emotion_theme"

EMOTION_MAP = {
    "joy": "Alegría",
    "sadness": "Tristeza",
    "anger": "Ira",
    "fear": "Miedo",
    "trust": "Confianza",
    "surprise": "Sorpresa"
}

THEME_MAP = {
    "love": "amor",
    "faith": "fe",
    "hope": "esperanza",
    "forgiveness": "perdón",
    "fear": "miedo"
}

# Invert for comparison
INV_EMOTION_MAP = {v.lower(): k for k, v in EMOTION_MAP.items()}
INV_THEME_MAP = {v.lower(): k for k, v in THEME_MAP.items()}


## 🧪 2. Load & Compare One Example Book (e.g., Genesis)

In [ ]:
book = "1_genesis"
en_file = EN_DIR / f"{book}_emotion_theme.csv"
es_file = ES_DIR / f"{book}_emotion_theme.csv"

df_en = pd.read_csv(en_file)
df_es = pd.read_csv(es_file)

assert len(df_en) == len(df_es)


## 🧠 3. Compare Emotions

In [ ]:
# Map Spanish → English
df_es["emotion_en"] = df_es["emotion"].str.lower().map(INV_EMOTION_MAP)
emotion_matches = df_en["emotion"].str.lower() == df_es["emotion_en"]
emotion_accuracy = emotion_matches.mean()

print(f"🎭 Emotion agreement: {emotion_accuracy:.2%}")


## 🧩 4. Compare Themes (Multi-label, unordered)

In [ ]:
def normalize_themes(series, inverse_map):
    def map_themes(row):
        if pd.isna(row): return set()
        return set(inverse_map.get(x.strip().lower(), x.strip().lower()) for x in row.split(";"))
    return series.apply(map_themes)

en_themes = normalize_themes(df_en["theme"], {})
es_themes = normalize_themes(df_es["theme"], INV_THEME_MAP)

theme_match = (en_themes == es_themes)
theme_overlap = [
    len(en & es) / max(len(en | es), 1)
    for en, es in zip(en_themes, es_themes)
]

print(f"🧠 Exact theme match: {theme_match.mean():.2%}")
print(f"🔁 Avg. theme overlap: {sum(theme_overlap)/len(theme_overlap):.2%}")


## 📊 5. Show Mismatches (Optional Debug View)

In [ ]:
mismatched = df_en[~emotion_matches].copy()
mismatched["es_emotion"] = df_es.loc[~emotion_matches, "emotion"]
mismatched[["chapter", "verse", "text", "emotion", "es_emotion"]].head(10)

## ✅ Conclusion
This notebook helps you inspect the thematic and emotional annotation process, for the English and the Spanish version.